In [48]:
#First recreate the data from the preliminary analysis
import geopandas as gpd
import pandas as pd


# Load and join GMCA housing, industrial and office supply data
housing_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Housing Land Supply GIS.shp")
industrial_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Industrial-warehousing Land Supply GIS.shp")
offices_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Offices Land Supply GIS.shp")
total_supply_gdf = pd.concat(
    [housing_supply_gdf, industrial_supply_gdf, offices_supply_gdf]
)

# Load and tidy GMEU Sites of Biological Importance data
sbi_gdf = gpd.read_file("data/gmeu_data/gm_sbi.shp")
sbi_gdf["Category"] = "Site of Biological Importance"
sbi_gdf = sbi_gdf.rename(columns = {"district": "LAName", "site_nam": "SiteRef"})

# Join GMCA and GMEU data
full_data_gdf = pd.concat(
    [total_supply_gdf, sbi_gdf[["SiteRef", "LAName", "Category", "geometry"]]]
)

#Use geopandas to get centroids of all the sites
full_data_gdf["centroid"] = full_data_gdf.centroid
full_data_gdf["ref"] = range(len(full_data_gdf))

#Split into sites of biological importance and non-biological importance
sbi = full_data_gdf[full_data_gdf["Category"] == "Site of Biological Importance"]
non_sbi = full_data_gdf[full_data_gdf["Category"] != "Site of Biological Importance"]

#Find the number of new developments less than 1km away for each SBI
sbinames = list(sbi["SiteRef"]) 
indexes = list(sbi["ref"])


#list of all the sbis
distances = list()
less_than_1km = list() #creating empty lists to add to data frame

for x in sbi["centroid"]: #loop through each sbi
    y = non_sbi["centroid"].distance(x) #find all the distances of developments to centroid
    for distance in y: #filter for less than 1km away
            if distance <1000:
                distances.append(distance)
    r = len(distances)    #find no. developments less than 1km away to each sbi
    less_than_1km.append(r)
    distances = list()

Dev_1km = pd.DataFrame({'SiteRef':sbinames, 'No. Sites in 1km': less_than_1km, 'ref': indexes}) #create dataframe of sbi and no. developments     


Dev_1km


,SiteRef,No. Sites in 1km,ref
0,Big Wood,0,4357
1,Winstanley Hall Woods,1,4358
2,Ackhurst Lane Sand Workings,3,4359
3,Abbey Lakes,6,4360
4,Wetland by M6,1,4361
...,...,...,...
531,Mill Race & Pasture at Haughton Dale,8,4888
532,Three Sisters,2,4889
533,Nan Nook Wood,7,4890
534,Big Wood,10,4891


In [49]:
#Obtaining the joined list
boundaries = gpd.read_file("data/boundaries/LAD_DEC_24_UK_BUC.shp")

pd.merge(full_data_gdf, boundaries, left_on = "LAName", right_on = "LAD24NM", how = "left")

joined_one = pd.merge(full_data_gdf, boundaries, left_on = "LAName", right_on = "LAD24NM", how = "left")

In [50]:
joined_one = joined_one.drop(columns=["LAD24NM","LAD24NMW","BNG_E","BNG_N","LONG","LAT","GlobalID","LAD24CD"])

In [51]:
joined_one.rename(columns={'geometry_x': 'SBI_Location', 'geometry_y': 'Local Authority'}, inplace=True)

In [52]:
joined_one


,Category,LAName,SiteRef,SBI_Location,centroid,ref,Local Authority
0,Housing,Bury,HL/0414/00,"POLYGON ((379112.8 410310.51, 379117.56 410306...",POINT (379114.012 410213.116),0,"POLYGON ((382118.504 413135.864, 383878.498 41..."
1,Housing,Bury,HL/1172/00,"POLYGON ((377973.95 414475.81, 377979.24 41446...",POINT (377947.835 414384.462),1,"POLYGON ((382118.504 413135.864, 383878.498 41..."
2,Housing,Bury,HL/1981/00,"POLYGON ((380839 413359.78, 380838.6 413361.95...",POINT (380885.718 413390.494),2,"POLYGON ((382118.504 413135.864, 383878.498 41..."
3,Housing,Bury,HL/2004/00,"POLYGON ((378834 407567.1, 378840 407566.81, 3...",POINT (378852.408 407559.6),3,"POLYGON ((382118.504 413135.864, 383878.498 41..."
4,Housing,Bury,HL/2127/00,"POLYGON ((379113.7 410329.59, 379114.66 410328...",POINT (379117.39 410313.26),4,"POLYGON ((382118.504 413135.864, 383878.498 41..."
...,...,...,...,...,...,...,...
4888,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),4888,"POLYGON ((402525.503 401459.003, 400905.596 39..."
4889,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),4889,"POLYGON ((358668.063 410971.684, 361851.801 40..."
4890,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),4890,"POLYGON ((384375.197 404714.801, 385603.299 40..."
4891,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),4891,"POLYGON ((384375.197 404714.801, 385603.299 40..."


In [53]:
pd.merge(joined_one, Dev_1km, left_on = "ref", right_on = "ref")
joined_two = pd.merge(joined_one, Dev_1km, left_on = "ref", right_on = "ref")

In [54]:
joined_two

,Category,LAName,SiteRef_x,SBI_Location,centroid,ref,Local Authority,SiteRef_y,No. Sites in 1km
0,Site of Biological Importance,Wigan,Big Wood,"POLYGON ((353649.752 410536.46, 353660.55 4105...",POINT (353331.975 410159.462),4357,"POLYGON ((358668.063 410971.684, 361851.801 40...",Big Wood,0
1,Site of Biological Importance,Wigan,Winstanley Hall Woods,"MULTIPOLYGON (((354300.461 403639.988, 354318....",POINT (354500.103 403252.365),4358,"POLYGON ((358668.063 410971.684, 361851.801 40...",Winstanley Hall Woods,1
2,Site of Biological Importance,Wigan,Ackhurst Lane Sand Workings,"POLYGON ((354320.268 407088.639, 354329.097 40...",POINT (354217.366 407048.76),4359,"POLYGON ((358668.063 410971.684, 361851.801 40...",Ackhurst Lane Sand Workings,3
3,Site of Biological Importance,Wigan,Abbey Lakes,"POLYGON ((352622.159 404628.83, 352625.623 404...",POINT (352671.79 404738.385),4360,"POLYGON ((358668.063 410971.684, 361851.801 40...",Abbey Lakes,6
4,Site of Biological Importance,Wigan,Wetland by M6,"MULTIPOLYGON (((354408.45 411806.416, 354430.9...",POINT (354504.673 411797.58),4361,"POLYGON ((358668.063 410971.684, 361851.801 40...",Wetland by M6,1
...,...,...,...,...,...,...,...,...,...
531,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),4888,"POLYGON ((402525.503 401459.003, 400905.596 39...",Mill Race & Pasture at Haughton Dale,8
532,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),4889,"POLYGON ((358668.063 410971.684, 361851.801 40...",Three Sisters,2
533,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),4890,"POLYGON ((384375.197 404714.801, 385603.299 40...",Nan Nook Wood,7
534,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),4891,"POLYGON ((384375.197 404714.801, 385603.299 40...",Big Wood,10


In [55]:
joined_two_At_Risk = joined_two[joined_two["No. Sites in 1km"]>14]

In [56]:
joined_two_At_Risk

,Category,LAName,SiteRef_x,SBI_Location,centroid,ref,Local Authority,SiteRef_y,No. Sites in 1km
5,Site of Biological Importance,Wigan,Ponds at Primrose Lane,"POLYGON ((355587.865 410547.072, 355609.634 41...",POINT (355707.163 410520.464),4362,"POLYGON ((358668.063 410971.684, 361851.801 40...",Ponds at Primrose Lane,15
19,Site of Biological Importance,Wigan,Meadow near Kirkless Hall,"POLYGON ((359875.249 406253.168, 359867.86 406...",POINT (360000.375 406294.412),4376,"POLYGON ((358668.063 410971.684, 361851.801 40...",Meadow near Kirkless Hall,15
24,Site of Biological Importance,Wigan,Firs Park,"POLYGON ((364518.048 400681.669, 364529.747 40...",POINT (364488.113 400600.946),4381,"POLYGON ((358668.063 410971.684, 361851.801 40...",Firs Park,16
43,Site of Biological Importance,Bolton,Bradford Reservoir,"POLYGON ((372273.276 407679.975, 372262.476 40...",POINT (372190.425 407595.22),4400,"POLYGON ((375025.76 414990.104, 374998.996 413...",Bradford Reservoir,19
51,Site of Biological Importance,Bury,Wetland near Radcliffe,"POLYGON ((378742.219 407785.863, 378694.906 40...",POINT (378744.817 407815.339),4408,"POLYGON ((382118.504 413135.864, 383878.498 41...",Wetland near Radcliffe,37
66,Site of Biological Importance,Salford,Bridgewater Canal,"POLYGON ((376376.965 399472.474, 376398.202 39...",POINT (376333.494 398534.86),4423,"POLYGON ((377681.197 404158.1, 379515.5 403434...",Bridgewater Canal,21
67,Site of Biological Importance,Salford,Three Sisters,"POLYGON ((377597.954 399827.739, 377624.402 39...",POINT (377636.164 399700.487),4424,"POLYGON ((377681.197 404158.1, 379515.5 403434...",Three Sisters,15
68,Site of Biological Importance,Salford,Salford Quays (North),"POLYGON ((380483.893 397023.667, 380504.568 39...",POINT (380794.224 396941.683),4425,"POLYGON ((377681.197 404158.1, 379515.5 403434...",Salford Quays (North),33
70,Site of Biological Importance,Salford,Blackleach Reservoir,"POLYGON ((373653.338 403890.027, 373654.283 40...",POINT (373993.954 404068.892),4427,"POLYGON ((377681.197 404158.1, 379515.5 403434...",Blackleach Reservoir,15
80,Site of Biological Importance,Manchester,Clayton Vale,"MULTIPOLYGON (((388092.022 399403.207, 388099....",POINT (388277.776 399437.268),4437,"POLYGON ((384375.197 404714.801, 385603.299 40...",Clayton Vale,24


In [57]:
len(joined_two_At_Risk)

45

In [58]:
joined_two

,Category,LAName,SiteRef_x,SBI_Location,centroid,ref,Local Authority,SiteRef_y,No. Sites in 1km
0,Site of Biological Importance,Wigan,Big Wood,"POLYGON ((353649.752 410536.46, 353660.55 4105...",POINT (353331.975 410159.462),4357,"POLYGON ((358668.063 410971.684, 361851.801 40...",Big Wood,0
1,Site of Biological Importance,Wigan,Winstanley Hall Woods,"MULTIPOLYGON (((354300.461 403639.988, 354318....",POINT (354500.103 403252.365),4358,"POLYGON ((358668.063 410971.684, 361851.801 40...",Winstanley Hall Woods,1
2,Site of Biological Importance,Wigan,Ackhurst Lane Sand Workings,"POLYGON ((354320.268 407088.639, 354329.097 40...",POINT (354217.366 407048.76),4359,"POLYGON ((358668.063 410971.684, 361851.801 40...",Ackhurst Lane Sand Workings,3
3,Site of Biological Importance,Wigan,Abbey Lakes,"POLYGON ((352622.159 404628.83, 352625.623 404...",POINT (352671.79 404738.385),4360,"POLYGON ((358668.063 410971.684, 361851.801 40...",Abbey Lakes,6
4,Site of Biological Importance,Wigan,Wetland by M6,"MULTIPOLYGON (((354408.45 411806.416, 354430.9...",POINT (354504.673 411797.58),4361,"POLYGON ((358668.063 410971.684, 361851.801 40...",Wetland by M6,1
...,...,...,...,...,...,...,...,...,...
531,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),4888,"POLYGON ((402525.503 401459.003, 400905.596 39...",Mill Race & Pasture at Haughton Dale,8
532,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),4889,"POLYGON ((358668.063 410971.684, 361851.801 40...",Three Sisters,2
533,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),4890,"POLYGON ((384375.197 404714.801, 385603.299 40...",Nan Nook Wood,7
534,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),4891,"POLYGON ((384375.197 404714.801, 385603.299 40...",Big Wood,10


In [62]:
#this gives the number of sbi's in eahc local authority that is "at risk" according to the current threshhold.

joined_two_At_Risk.groupby("LAName")[["No. Sites in 1km"]].sum()

,No. Sites in 1km
LAName,
Bolton,99
Bury,138
Manchester,207
Oldham,67
Rochdale,40
Salford,102
Tameside,70
Trafford,144
Wigan,62
